In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import warnings

warnings.filterwarnings("ignore")

# MySQL connection parameters
user = 'root'
password = 'admin'
MYSQL_HOST = 'localhost'
database = 'weather_db'

# Create an engine for the MySQL connection
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{MYSQL_HOST}/{database}')

# Load data from the database
query = """
    SELECT city_name, date, max_temperature, min_temperature, max_humidity, max_wind_speed
    FROM WeatherData
"""
df = pd.read_sql(query, con=engine)

# Data cleaning and processing
df.dropna(inplace=True)
df['max_temperature'] = pd.to_numeric(df['max_temperature'], errors='coerce')
df['min_temperature'] = pd.to_numeric(df['min_temperature'], errors='coerce')
df['max_humidity'] = pd.to_numeric(df['max_humidity'], errors='coerce')
df['max_wind_speed'] = pd.to_numeric(df['max_wind_speed'], errors='coerce')
df['date'] = pd.to_datetime(df['date'])
df.dropna(inplace=True)

# Sort values
df.sort_values(['date'], inplace=True)


In [2]:
df

,city_name,date,max_temperature,min_temperature,max_humidity,max_wind_speed
83,pune,2024-07-18,24.3,23.4,97.0,24.3
5332,surat,2024-07-18,29.4,13.3,93.0,9.5
5513,morbi,2024-07-18,36.1,28.0,91.0,17.9
5694,delhi,2024-07-18,39.3,32.9,58.0,18.3
5875,Vadodara,2024-07-18,35.1,28.0,87.0,31.3
...,...,...,...,...,...,...
15697,mumbai,2024-11-20,28.4,21.5,81.0,9.7
15574,Weir,2024-11-20,14.0,8.2,61.0,29.1
15575,Pali,2024-11-20,22.6,12.0,72.0,11.3
15573,Weir,2024-11-20,18.8,8.2,61.0,29.1


In [3]:
df.iloc[-1]

city_name                        thane
date               2024-11-20 00:00:00
max_temperature                   27.8
min_temperature                   17.1
max_humidity                      99.0
max_wind_speed                     6.6
Name: 15698, dtype: object

In [4]:
from prophet import Prophet
import pandas as pd
import pickle

df = df
df['date'] = pd.to_datetime(df['date'])

# List of variables to forecast
variables = ['max_temperature', 'min_temperature', 'max_humidity', 'max_wind_speed']

for var in variables:
    # Prepare data for the current variable
    var_df = df[['date', var]].rename(columns={'date': 'ds', var: 'y'})
    
    # Initialize and train the model
    model = Prophet()
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.fit(var_df)
    
    # Save the model
    with open(f"{var}_model.pkl", 'wb') as file:
        pickle.dump(model, file)


10:46:52 - cmdstanpy - INFO - Chain [1] start processing
10:46:53 - cmdstanpy - INFO - Chain [1] done processing
10:46:53 - cmdstanpy - INFO - Chain [1] start processing
10:46:54 - cmdstanpy - INFO - Chain [1] done processing
10:46:54 - cmdstanpy - INFO - Chain [1] start processing
10:46:55 - cmdstanpy - INFO - Chain [1] done processing
10:46:55 - cmdstanpy - INFO - Chain [1] start processing
10:46:57 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
import pickle
from prophet import Prophet
import pandas as pd

# Sample DataFrame for user-provided last known values (example df)
user_input = df.iloc[-1]

# Extract the last row values for relevant columns
last_row_values = user_input[['max_temperature', 'min_temperature', 'max_humidity', 'max_wind_speed']].to_dict()

# Extract city_name from user input
city_name = user_input['city_name']

# Define the number of days to forecast
forecast_days = 7

# Dictionary to store predictions
predictions = {}

# For each weather variable, load the corresponding model and make predictions
for var, value in last_row_values.items():
    # Load the saved model for the variable
    with open(f"{var}_model.pkl", 'rb') as file:
        model = pickle.load(file)
    
    # Generate future dates for forecasting
    future = model.make_future_dataframe(periods=forecast_days)
    
    # Predict the future values
    forecast = model.predict(future)
    # Retrieve the last `forecast_days` predictions and rename 'yhat' to the variable name
    predictions[var] = forecast[['ds', 'yhat']].tail(forecast_days).rename(columns={'yhat': var})

# Initialize the combined DataFrame with one of the forecasts
combined_forecast_df = predictions['max_temperature']

# Merge the remaining forecasts on the 'ds' column
for var, forecast in predictions.items():
    if var != 'max_temperature':
        combined_forecast_df = combined_forecast_df.merge(forecast, on='ds')

# Add the city_name to the combined DataFrame
combined_forecast_df['city_name'] = city_name

# Replace negative values with NaN in only the forecasted columns, then apply forward-fill
forecast_columns = list(last_row_values.keys())
combined_forecast_df[forecast_columns] = combined_forecast_df[forecast_columns].applymap(lambda x: x if x >= 0 else None).ffill()

# Rearrange the DataFrame to match the requested format
final_df = combined_forecast_df[['ds', 'city_name', 'max_temperature', 'min_temperature', 'max_humidity', 'max_wind_speed']]

# Rename columns to match the requested format
final_df = final_df.rename(columns={
    'ds': 'date',
    'max_temperature': 'temperature',
    'max_humidity': 'humidity',
    'max_wind_speed': 'wind_speed'
})

# Display the final DataFrame
print(final_df)


        date city_name  temperature  min_temperature   humidity  wind_speed
0 2024-11-21     thane    29.978274        18.215282  82.480217    8.516297
1 2024-11-22     thane    29.878299        18.017124  82.549048    9.039429
2 2024-11-23     thane    29.454554        17.696951  82.946133    9.094316
3 2024-11-24     thane    28.886607        17.394494  82.776019    9.866458
4 2024-11-25     thane    28.482168        17.205812  82.367236   10.931722
5 2024-11-26     thane    28.031683        16.962652  82.181293   10.930599
6 2024-11-27     thane    27.844650        16.991456  82.153964   11.351718
